# Logging
Le canal de communication entre l'utilisateur et le serveur doit être sécurisé. (TLS ?)

## Enregistrement
- L'utilisateur génère aléatoirement un sel
- L'utilisateur hash son mot de passe avec **Argon2** : $hpwd = argon2(pwd, salt)$
- L'utilisateur envoie ses informations au serveur (qui vérifie si le nom existe déjà): $user_{\text{infos}} = (name, hpwd, salt)$

## Identification
- L'utilisateur envoie une requette au serveur avec son username
- Le serveur répond avec le sel associé
- L'utilisateur recalcul $hpwd = argon2(pwd, salt)$
- L'utilisateur transmets $(username, hpwd)$ au serveur
- Le serveur vérifie et si le $hpwd$ reçu correspond bien a celui stocké

## Changement de mot de passe
- L'utilisateur effectue la procédure d'identification
- L'utilisateur effectue la procédure d'enregistrement
- Le serveur vérifie si le nom existe et si l'utilisateur correspondant est bien connecté
- Le serveur remplace les anciennes valeurs $(hpwd, salt)$ par celles reçues


In [1]:
from nacl import pwhash, secret, utils
from lphelpers import tracer

tr = tracer(trace_level='DEBUG')

class UserInfos :
    def __init__(self, name, hpwd, salt):
        self.name = name
        self.hpwd = hpwd
        self.salt = salt
        self.isconnected = False

    def __str__(self):
        return f"{self.name}"

class Server : 
    def __init__(self, name='Server'):
        self.name = name
        self.users  = []

    def register(self, new_user) : 
        if all(new_user.name != user.name for user in self.users):
            self.users.append(new_user)
            tr.info(f"[{self}]: New user {new_user.name} was added!")

    def getmysalt(self, requester_name) :
        for user in self.users :
            if user.name == requester_name :
                return user.salt   
        tr.error(f"[{self}]: No user is register as : {requester_name}")
        return None
    
    def login(self, requester_name, requester_hpwd) :
        for user in self.users :
            if user.name == requester_name :
                if user.hpwd == requester_hpwd :
                    user.isconnected = True
                    tr.info(f'[{self}]: User {requester_name} is now connected!')
                    return
                else : 
                    tr.error(f'[{self}]: Wrong password!')
                    return
        tr.error(f"[{self}]: No user is register as : {requester_name}")
    
    def logout(self, requester_name) :
        for user in self.users:
            if user.name == requester_name:
                user.isconnected = False
                tr.info(f"[{self}]: {requester_name} has been logged out.")
                return
        tr.error(f"[{self}]: No user is registered as: {requester_name}")

    def update(self, requester):
        for user in self.users :
            if user.name == requester.name :
                if not user.isconnected :
                    tr.warn(f'[{self}]: Please login before!')
                    return
                user.hpwd = requester.hpwd
                user.salt = requester.salt
                tr.info(f"[{self}]: Password updated for {user.name}")
                return
        tr.error(f"[{self}]: No user is register as : {requester.name}")
    
    def remove(self, requester_name):
        self.users = [user for user in self.users if user.name != requester_name]
        tr.info(f"[{self}]: User {requester_name} was successfully removed!")

    def show_registered_users(self):
        for user in self.users :
            tr.info(user.name)

    def __str__(self):
        return f"{self.name}"
     
class User : 
    def __init__(self, name, password):
        self.name = name
        self.password = password
    
    def gen_public_infos(self):
        tr.debug(f'[{self.name}]: Drawing a random salt...')
        salt = utils.random(pwhash.argon2i.SALTBYTES)                               # Draw a random salt 
        tr.debug(f'[{self.name}]: Generating public informations...')
        hpwd = pwhash.argon2i.kdf(secret.SecretBox.KEY_SIZE, self.password, salt)   # Compute Hpwd
        return UserInfos(self.name, hpwd, salt)
    
    def register_on(self, server):
        public_infos = self.gen_public_infos()
        tr.debug(f'[{self.name}]: Request a registration on {server}')
        server.register(public_infos)

    def login_on(self, server):
        tr.debug(f'[{self.name}]: Getting salt from {server}')
        salt = server.getmysalt(self.name)                                          # Recover the salt
        tr.debug(f'[{self.name}]: Recomputing hpwd')
        hpwd = pwhash.argon2i.kdf(secret.SecretBox.KEY_SIZE, self.password, salt)   # Recompute hpwd
        tr.debug(f'[{self.name}]: Request a login on {server}')
        server.login(self.name, hpwd)                                               # Login on server

    def logout_from(self, server):
        tr.debug(f'[{self.name}]: Request a logout from {server}')
        server.logout(self.name)

    def change_password_on(self, server, new_password):
        self.password = new_password
        public_infos = self.gen_public_infos() # Generate new public infos using new pwd (new salt as well)
        tr.debug(f'[{self.name}]: Request a passord update on {server}')
        server.update(public_infos)

    def __str__(self):
        return f"{self.name}"

server = Server(name = 'ServerMSE')
transmitter = User(name = 'TransmitterMSE', password = b'password@transmitter')
transmitter.register_on(server)
transmitter.login_on(server)
transmitter.change_password_on(server, b'myNew@password')
transmitter.logout_from(server)

[TransmitterMSE]: Drawing a random salt...
[TransmitterMSE]: Generating public informations...
[TransmitterMSE]: Request a registration on ServerMSE
[ServerMSE]: New user TransmitterMSE was added!
[TransmitterMSE]: Getting salt from ServerMSE
[TransmitterMSE]: Recomputing hpwd
[TransmitterMSE]: Request a login on ServerMSE
[ServerMSE]: User TransmitterMSE is now connected!
[TransmitterMSE]: Drawing a random salt...
[TransmitterMSE]: Generating public informations...
[TransmitterMSE]: Request a passord update on ServerMSE
[ServerMSE]: Password updated for TransmitterMSE
[TransmitterMSE]: Request a logout from ServerMSE
[ServerMSE]: TransmitterMSE has been logged out.
